In [1]:
import os
import pandas as pd
import sqlalchemy as db
from dotenv import load_dotenv

In [2]:
load_dotenv()

# config and credentials
server = os.getenv('server')
database = os.getenv('database')
username = os.getenv('user')
password = os.getenv('password')

# connection
engine = db.create_engine(
    'mssql://{}:{}@{}/{}?driver=ODBC+Driver+18+for+SQL+Server'.format(
        username, password, server, database
    )
)

# establish connection
connection = engine.connect()

### Daily Distinct Cusips and Total Volume by Retail / Institunional grouping

In [3]:
def distinctCusips_totalVolume_by_retail_institutional(year_start, year_end, discard_nr):
    
    base_query = '''
        SELECT 
            TrdExctnDt,
            RetailThreshold,
            COUNT(DISTINCT CusipId) AS DistinctCusips,
            SUM(EntrdVolQt) AS TotalVolume
        FROM (
            SELECT
                TrdExctnDt,
                CASE WHEN EntrdVolQt < 100000 THEN 'R' ELSE 'IN' END AS RetailThreshold,
                CusipId,
                EntrdVolQt
            FROM
                Trace_filtered_withRatings
            WHERE
                TrdExctnDt >= '{}-01-1' AND TrdExctnDt < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    if discard_nr:
        base_query += 'AND RatingNum <> 0'
    
    base_query += '''
        ) A
        GROUP BY
            TrdExctnDt,
            RetailThreshold
        ORDER BY
            TrdExctnDt, 
            RetailThreshold
    '''
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [4]:
start, end = 2002, 2022

df = distinctCusips_totalVolume_by_retail_institutional(year_start=start, year_end=end, discard_nr=True)
df.to_csv('data/output/bonds/distinctCusips_totalVolume_by_retail_institutional_{}-{}.csv'.format(start, end), index=False)
df.head()

,TrdExctnDt,RetailThreshold,DistinctCusips,TotalVolume
0,2002-07-01,IN,991,4.468586e+11
1,2002-07-01,R,1309,6.363085e+09
2,2002-07-02,IN,1020,7.580465e+11
3,2002-07-02,R,1346,7.069888e+09
4,2002-07-03,IN,686,2.987376e+11


### Daily Distinct Cusips and Total Volume

In [5]:
def distinctCusips_totalVolume(year_start, year_end, discard_nr):
    
    base_query = '''
        SELECT 
            TrdExctnDt,
            COUNT(DISTINCT CusipId) AS DistinctCusips,
            SUM(EntrdVolQt) AS TotalVolume
        FROM
            Trace_filtered_withRatings
        WHERE
            TrdExctnDt >= '{}-01-1' AND TrdExctnDt < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    if discard_nr:
        base_query += 'AND RatingNum <> 0'
    
    base_query += '''
        GROUP BY
            TrdExctnDt
        ORDER BY
            TrdExctnDt
    '''
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [6]:
start, end = 2002, 2022

df = distinctCusips_totalVolume(year_start=start, year_end=end, discard_nr=True)
df.to_csv('data/output/bonds/distinctCusips_totalVolume_{}-{}.csv'.format(start, end), index=False)
df.head()

,TrdExctnDt,DistinctCusips,TotalVolume
0,2002-07-01,1797,4.532217e+11
1,2002-07-02,1844,7.651164e+11
2,2002-07-03,1480,3.036958e+11
3,2002-07-05,917,7.858672e+10
4,2002-07-08,1937,4.479530e+11


### Daily Total Volume by Side (Buy/Sell) 

In [7]:
def totalVolume_by_side(year_start, year_end, discard_nr):
    
    base_query = '''
        SELECT 
            TrdExctnDt,
            RptSideCd,
            SUM(EntrdVolQt) AS TotalVolume,
            COUNT(DISTINCT CusipId) AS DistinctCusips
        FROM
            Trace_filtered_withRatings
        WHERE
            TrdExctnDt >= '{}-01-1' AND TrdExctnDt < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    if discard_nr:
        base_query += 'AND RatingNum <> 0'
            
    base_query += '''
        GROUP BY
            TrdExctnDt, 
            RptSideCd
        ORDER BY
            TrdExctnDt, 
            RptSideCd
    '''
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [8]:
start, end = 2002, 2022

df = totalVolume_by_side(year_start=start, year_end=end, discard_nr=True)
df.to_csv('data/output/bonds/totalVolume_by_side_{}-{}.csv'.format(start, end), index=False)
df.head()

,TrdExctnDt,RptSideCd,TotalVolume,DistinctCusips
0,2002-07-01,B,2.043281e+11,1286
1,2002-07-01,S,2.488936e+11,1229
2,2002-07-02,B,4.062710e+11,1321
3,2002-07-02,S,3.588455e+11,1335
4,2002-07-03,B,1.230611e+11,938


### Industry analysis on Cusips and Issuers

In [9]:
def industry_analysis_on_cusips_issuers(year_start, year_end, discard_nr):
    
    base_query = '''
        SELECT
            IndustryCode,
            DistinctCusips,
            DistinctIssuers
        FROM (
            SELECT 
                IndustryCode,
                COUNT(DISTINCT CusipId) AS DistinctCusips,
                COUNT(DISTINCT IssuerID) AS DistinctIssuers
            FROM
                Trace_filtered_withRatings
            WHERE
                TrdExctnDt >= '{}-01-1' AND TrdExctnDt < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    if discard_nr:
        base_query += 'AND RatingNum <> 0'
    
    base_query += '''
            GROUP BY
                IndustryCode 
        ) A
        ORDER BY
            IndustryCode
    '''
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [10]:
start, end = 2002, 2022

df = industry_analysis_on_cusips_issuers(year_start=start, year_end=end, discard_nr=True)
df.to_csv('data/output/bonds/industry_analysis_on_cusips_issuers_{}-{}.csv'.format(start, end), index=False)
df.head()

,IndustryCode,DistinctCusips,DistinctIssuers
0,10,5759,852
1,11,1407,233
2,12,1575,294
3,13,82,11
4,14,1087,146


### Daily Distinct Cusips and Total Volume by Issuer

In [11]:
def distinctCusips_totalVolume_by_issuer(year_start, year_end, discard_nr):
    
    base_query = '''
        SELECT
            TrdExctnDt,
            IssuerId,
            COUNT(DISTINCT CusipId) AS DistinctCusips,
            SUM(EntrdVolQt) AS TotalVolume
        FROM 
            Trace_filtered_withRatings
        WHERE
            TrdExctnDt >= '{}-01-1' AND TrdExctnDt < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    if discard_nr:
        base_query += 'AND RatingNum <> 0'
    
    base_query += '''
        GROUP BY
            TrdExctnDt, 
            IssuerId
        ORDER BY
            TrdExctnDt, 
            IssuerId
    '''
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [12]:
start, end = 2002, 2022

df = distinctCusips_totalVolume_by_issuer(year_start=start, year_end=end, discard_nr=True)
df.to_csv('data/output/bonds/distinctCusips_totalVolume_by_issuer_{}-{}.csv'.format(start, end), index=False)
df.head()

,TrdExctnDt,IssuerId,DistinctCusips,TotalVolume
0,2002-07-01,11,5,5.214120e+08
1,2002-07-01,26,13,1.290441e+09
2,2002-07-01,34,1,5.000000e+06
3,2002-07-01,41,3,2.195680e+08
4,2002-07-01,74,1,8.400000e+06


### Daily Distinct Cusips by Rating

In [13]:
def distinctCusips_by_rating(year_start, year_end, discard_nr):
    
    base_query = '''
        SELECT
            TrdExctnDt,
            RatingNum,
            COUNT(DISTINCT CusipId) AS DistinctCusips
        FROM 
            Trace_filtered_withRatings A
        WHERE
            A.TrdExctnDt >= '{}-01-1' AND A.TrdExctnDt < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    if discard_nr:
        base_query += 'AND RatingNum <> 0'
    
    base_query += '''
        GROUP BY
            TrdExctnDt,
            RatingNum
        ORDER BY
            TrdExctnDt,
            RatingNum
    '''
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [14]:
start, end = 2002, 2022

df = distinctCusips_by_rating(year_start=start, year_end=end, discard_nr=True)
df.to_csv('data/output/bonds/distinctCusips_by_rating_{}-{}.csv'.format(start, end), index=False)
df.head()

,TrdExctnDt,RatingNum,DistinctCusips
0,2002-07-01,1.0,70
1,2002-07-01,2.0,26
2,2002-07-01,3.0,74
3,2002-07-01,4.0,301
4,2002-07-01,5.0,395


### Distinct Cusips per Maturity Band

In [15]:
def distinctCusips_by_maturity(year_start, year_end, discard_nr):
    
    base_query = '''
        SELECT
            MaturityBand,
            COUNT(DISTINCT CusipId) AS DistinctCusips
        FROM (
            SELECT
                CASE 
                    WHEN ABS(DATEDIFF(DAY, Maturity, OfferingDate)) * 1.0 / 360 < 5 THEN 1
                    WHEN ABS(DATEDIFF(DAY, Maturity, OfferingDate)) * 1.0 / 360 < 15 THEN 2
                    ELSE 3
                END AS MaturityBand,
                CusipId
            FROM
                Trace_filtered_withRatings
            WHERE
                TrdExctnDt >= '{}-01-1' AND TrdExctnDt < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    if discard_nr:
        base_query += 'AND RatingNum <> 0'
    
    base_query += '''
        ) A
        GROUP BY
            MaturityBand
        ORDER BY
            MaturityBand
    '''
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [16]:
start, end = 2002, 2022

df = distinctCusips_by_maturity(year_start=start, year_end=end, discard_nr=True)
df.to_csv('data/output/bonds/distinctCusips_by_maturity_{}-{}.csv'.format(start, end), index=False)
df.head()

,MaturityBand,DistinctCusips
0,1,4257
1,2,17268
2,3,4489


### Distinct Cusips per Investment Grade

In [17]:
def distinctCusips_by_investmentGrade(year_start, year_end):
    
    base_query = '''
        SELECT
            InvestmentGrade,
            COUNT(DISTINCT CusipId) AS DistinctCusips
        FROM (
            SELECT
                CusipId,
                CASE
                    WHEN RatingNum = 0 THEN 'NR'
                    WHEN RatingNum < 11 THEN 'Y'
                    ELSE 'N'
                END AS InvestmentGrade
            FROM 
                Trace_filtered_withRatings
            WHERE
                TrdExctnDt >= '{}-01-1' AND TrdExctnDt < '{}-01-01'
        ) A
        GROUP BY
            InvestmentGrade
        ORDER BY
            InvestmentGrade
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [18]:
start, end = 2002, 2022

df = distinctCusips_by_investmentGrade(year_start=start, year_end=end)
df.to_csv('data/output/bonds/distinctCusips_by_investmentGrade_{}-{}.csv'.format(start, end), index=False)
df.head()

,InvestmentGrade,DistinctCusips
0,N,5769
1,NR,37917
2,Y,22491


### Distinct Trading Days

In [19]:
def distinctTradingDays(year_start, year_end, discard_nr):
    
    base_query = '''
        SELECT
            COUNT(DISTINCT TrdExctnDt)
        FROM
            Trace_filtered_withRatings
        WHERE
            TrdExctnDt >= '{}-01-1' AND TrdExctnDt < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    if discard_nr:
        base_query += 'AND RatingNum <> 0'
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [20]:
start, end = 2002, 2022 # MIN = 2002-07-01 | MAX = 2022-09-30

df = distinctTradingDays(year_start=start, year_end=end, discard_nr=True)
df.to_csv('data/output/bonds/distinctTradingDays_{}-{}.csv'.format(start, end), index=False)
df.head()

,
0,5099
